In [1]:
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from modeling.models.bethge import BethgeModel
from modeling.train_utils import array_to_dataloader
from scipy.stats import spearmanr
from scripts.visualize_SharedCore import visualize_target
from scipy import optimize
import os
import cv2

In [14]:
site = 'm2s1'
nb_validation_samples = 1000
nb_training_samples = 49000

batch_size = 2048

train_y = np.load('A:/school/College_Junior/research/CNN_Tang_project/data/Processed_Tang_data/all_sites_data_prepared/New_response_data/trainRsp_'+site+'.npy')
num_neurons = train_y.shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

channels = 256
num_layers = 9
input_size = 50
output_size = num_neurons
first_k = 9
later_k = 3
pool_size = 2
factorized = True

num_maps = 1

net = BethgeModel(channels=channels, num_layers=num_layers, input_size=input_size,
                  output_size=output_size, first_k=first_k, later_k=later_k,
                  input_channels=1, pool_size=pool_size, factorized=True,
                  num_maps=num_maps).cuda()

net.to(device)
net.load_state_dict(torch.load('A:/school/College_Junior/research/CNN_Tang_project/saved_models/Sample_size_test_models/9_10_'+site+'_shared_core_256_9'))

fc_layer = net.fc[0].bank[0]

In [3]:
feature_w = fc_layer.weight_feature.data
spatial_w = fc_layer.weight_spatial.data
spatial_w_np = spatial_w.detach().cpu().numpy()

for i, w in enumerate(spatial_w):
    w[:8,:] = 0
    w[:,13:] = 0
    w[13:,:] = 0
    w[:,:8] = 0
    spatial_w[i] = w

net.fc[0].bank[0].weight_spatial.data = spatial_w

In [4]:
def make_dir_try(dir):
    try:
        os.makedirs(dir, exist_ok=True)
        print("Directory '%s' created successfully" % dir)
    except OSError as error:
        print("Directory '%s' can not be created" % dir)
directory = 'A:/school/College_Junior/research/CNN_Tang_project/analysis/Visualization/reconstruction_'+str(channels)+'_'+str(num_layers)
make_dir_try(directory)

Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/Visualization/reconstruction_256_9' created successfully


In [15]:
val_img = np.load('A:/school/College_Junior/research/CNN_Tang_project/data/Processed_Tang_data/all_sites_data_prepared/pics_data/val_img_m2s1.npy')
val_rsp = np.load('A:/school/College_Junior/research/CNN_Tang_project/data/Processed_Tang_data/all_sites_data_prepared/New_response_data/valRsp_m2s1.npy')
net.eval()
with torch.no_grad():
    img = np.reshape(val_img[0],(1,1,50,50))
    img = torch.FloatTensor(img).to(device)
    target = net(img)
visualize_target(target, input_size, net, directory, save=True)

BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


Lossy conversion from float32 to uint8. Range [0.0, 256.0]. Convert image to uint8 prior to saving to suppress this warning.
